# 3. 擴展鏈 (Chains Extended) - Gemini 版本

本範例展示如何在鏈中加入自定義處理步驟，建立一個智能郵件回覆系統，包含格式化和品質檢查。

## 學習重點
- 使用 RunnableLambda 添加自定義邏輯
- 學習如何擴展基本的鏈功能
- 理解鏈的模組化特性
- 掌握多步驟處理的技巧


In [1]:
# 導入必要的套件
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda
from langchain_google_genai import ChatGoogleGenerativeAI

# 載入環境變數
load_dotenv()

# 建立 Gemini 模型
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")


E0000 00:00:1759302825.518008   98830 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [2]:
# 定義提示模板 - 建立郵件回覆系統
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一個專業的客服代表，負責回覆客戶郵件。請用友善、專業的語調回覆。"),
        ("human", "請回覆這封客戶郵件：\n\n{email_content}"),
    ]
)


In [3]:
# 定義自定義處理步驟
# 使用 RunnableLambda 來添加自定義邏輯

# 步驟 1: 格式化郵件回覆（添加標題和簽名）
def format_email_reply(reply):
    """格式化郵件回覆，添加專業的郵件格式"""
    formatted_reply = f"""
親愛的客戶，

{reply}

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
---
回覆時間：{__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M')}
"""
    return formatted_reply.strip()

format_email = RunnableLambda(format_email_reply)

# 步驟 2: 添加品質檢查和統計資訊
def add_quality_check(formatted_reply):
    """添加品質檢查和統計資訊"""
    word_count = len(formatted_reply.split())
    char_count = len(formatted_reply)
    
    quality_info = f"""
📊 回覆統計：
- 字數：{word_count} 字
- 字元數：{char_count} 字元
- 回覆品質：{'✅ 良好' if word_count >= 50 else '⚠️ 建議增加內容'}

{formatted_reply}
"""
    return quality_info

quality_check = RunnableLambda(add_quality_check)

print("自定義處理步驟已定義：")
print("1. format_email: 格式化郵件回覆")
print("2. quality_check: 添加品質檢查和統計資訊")


自定義處理步驟已定義：
1. format_email: 格式化郵件回覆
2. quality_check: 添加品質檢查和統計資訊


In [4]:
# 建立擴展的鏈
# 鏈的流程：Prompt Template → LLM → Output Parser → 郵件格式化 → 品質檢查
chain = prompt_template | model | StrOutputParser() | format_email | quality_check

print("擴展鏈已建立完成！")
print("鏈的結構：Prompt Template → LLM → Output Parser → 郵件格式化 → 品質檢查")


擴展鏈已建立完成！
鏈的結構：Prompt Template → LLM → Output Parser → 郵件格式化 → 品質檢查


In [5]:
# 執行擴展鏈 - 測試郵件回覆系統
customer_email = """
您好，

我最近購買了貴公司的產品，但是發現包裝有損壞，而且產品功能與說明書不符。
請問可以退貨或換貨嗎？我已經購買一週了。

期待您的回覆。

王小明
"""

result = chain.invoke({"email_content": customer_email})

# 輸出結果
print("=" * 60)
print("智能郵件回覆系統處理結果：")
print("=" * 60)
print(result)


智能郵件回覆系統處理結果：

📊 回覆統計：
- 字數：27 字
- 字元數：527 字元
- 回覆品質：⚠️ 建議增加內容

親愛的客戶，

親愛的王小明先生您好，

感謝您來信告知產品狀況。

對於您收到的產品包裝有損壞，以及產品功能與說明書不符的情形，我們深感抱歉，這絕非我們希望提供給客戶的體驗。

關於您提出的退貨或換貨需求，我們當然樂意為您處理。請您放心，由於產品本身有瑕疵及不符的情形，即使已購買一週，我們也會依照公司的退換貨政策為您妥善處理。

為了能更快速地協助您辦理退換貨手續，懇請您提供以下資訊：

1.  您的訂單編號或購買證明（例如：發票號碼、網路訂單截圖等）。
2.  產品的名稱及型號。
3.  您希望選擇退貨（全額退款）或是換貨（更換一個全新的同款產品）。
4.  您方便的聯絡電話，以便我們進一步與您確認細節。

收到您的資訊後，我們會立即為您安排後續的處理方式，例如：派員到府取件、提供寄送方式等。

再次為這次不愉快的購物經驗向您致歉，我們非常重視每一位客戶的意見，並會將此情況反映給相關部門，以避免類似問題再次發生。

期待您的回覆，並期盼能有機會為您提供更好的服務。

祝您有美好的一天！

[您的公司名稱] 客服團隊 敬上

感謝您的來信，如有其他問題請隨時聯繫我們。

此致
客服團隊
---
回覆時間：2025-10-01 15:14



## 💡 重點說明

### 擴展鏈的優勢

1. **模組化設計**: 每個處理步驟都是獨立的，可以輕鬆添加、移除或修改
2. **可重用性**: 自定義的處理步驟可以在不同的鏈中重複使用
3. **靈活性**: 可以根據需求組合不同的處理步驟

### RunnableLambda 的使用

```python
# 基本語法
custom_step = RunnableLambda(lambda x: 自定義處理函數(x))

# 實際範例
format_email = RunnableLambda(format_email_reply)
quality_check = RunnableLambda(add_quality_check)
```

### 鏈的組合方式

```python
# 使用管道符號串聯多個步驟
chain = (
    prompt_template 
    | model 
    | StrOutputParser() 
    | format_email 
    | quality_check
)
```

## 🔧 實際應用場景

- **客服系統**: 自動回覆客戶郵件
- **內容管理**: 格式化生成內容
- **品質控制**: 自動檢查內容品質
- **報告生成**: 添加統計資訊和格式
